# NE551 Stopping Power Assignment #

## Using the stopping power equations for *heavy ions*, develop a code that will: ##

### Calculate the collisional mass stopping power of a charged particle (heavy ions with a given charge, Z) and kinetic energy through the materials listed below. For this assignment, heavy ions are any charged particle with mass greater than or equal to the proton ###

In [1]:
%%javascript
MathJax.Extension["TeX/cancel"]={version:"2.4.0",ALLOWED:{color:1,mathcolor:1,background:1,mathbackground:1,padding:1,thickness:1}};MathJax.Hub.Register.StartupHook("TeX Jax Ready",function(){var c=MathJax.InputJax.TeX,a=MathJax.ElementJax.mml,b=MathJax.Extension["TeX/cancel"];b.setAttributes=function(h,e){if(e!==""){e=e.replace(/ /g,"").split(/,/);for(var g=0,d=e.length;g<d;g++){var f=e[g].split(/[:=]/);if(b.ALLOWED[f[0]]){if(f[1]==="true"){f[1]=true}if(f[1]==="false"){f[1]=false}h[f[0]]=f[1]}}}return h};c.Definitions.Add({macros:{cancel:["Cancel",a.NOTATION.UPDIAGONALSTRIKE],bcancel:["Cancel",a.NOTATION.DOWNDIAGONALSTRIKE],xcancel:["Cancel",a.NOTATION.UPDIAGONALSTRIKE+" "+a.NOTATION.DOWNDIAGONALSTRIKE],cancelto:"CancelTo"}},null,true);c.Parse.Augment({Cancel:function(e,g){var d=this.GetBrackets(e,""),f=this.ParseArg(e);var h=b.setAttributes({notation:g},d);this.Push(a.menclose(f).With(h))},CancelTo:function(e,g){var i=this.ParseArg(e),d=this.GetBrackets(e,""),f=this.ParseArg(e);var h=b.setAttributes({notation:a.NOTATION.UPDIAGONALSTRIKE+" "+a.NOTATION.UPDIAGONALARROW},d);i=a.mpadded(i).With({depth:"-.1em",height:"+.1em",voffset:".1em"});this.Push(a.msup(a.menclose(f).With(h),i))}});MathJax.Hub.Startup.signal.Post("TeX cancel Ready")});MathJax.Ajax.loadComplete("[MathJax]/extensions/TeX/cancel.js");

<IPython.core.display.Javascript object>

In [9]:
import pyne.data as data
import math
import scipy.constants as const
import json

Equation 2.22 from Anderson: 

\begin{equation}
S_c = 4 z^2 \pi r_0^2 m_e \cancel{c^2} N_A \frac{Z}{M_m} \rho \frac{\left( T + M \cancel{c^2} \right) ^2}{T \left( T + 2 M \cancel{c^2} \right) } \ln \left( \frac{2 m_e \cancel{c^2} T \left( T + 2 M \cancel{c^2} \right) }{I \left( T + M \cancel{c^2} \right)^2} \right)
\end{equation}
This is equivalent to:
\begin{equation}
S_c = 4 z^2 \pi r_0^2 \frac{m_e \cancel{c^2} }{\beta^2} \frac{N_A \rho}{M_m} Z \ln \left( \frac{2 m_e \cancel{c^2} \gamma^2 \beta^2}{I} \right)
\end{equation}
To calculate the collisional \textit{mass} stopping power, divide this by $ \rho $:
\begin{equation}
S_c = 4 z^2 \pi r_0^2 \frac{m_e}{\beta^2} \frac{N_A}{M_m} Z \ln \left( \frac{2 m_e \gamma^2 \beta^2}{I} \right)
\end{equation}

First, import useful constants (polluting the namespace):

In [10]:
pi = math.pi
r_e = const.value('classical electron radius') * 100.0 # Convert 1m = 100cm
m_e = const.value('electron mass energy equivalent in MeV')
N_A = const.value('Avogadro constant')
m_to_e = const.physical_constants['atomic mass constant energy equivalent in MeV'][0]

In [11]:
def I(material):
    """ Looks up the Adjusted Average Excitation Energy if Z<=13. Otherwise calculates it
        Output units are in MeV
    """
    # For now, specify material with Z value
    Z = material
    
    # Z must be an integer
    assert type(Z) == type(4)
    
    # Lookup table for Z<=13. Key is the "Z" of the material. Values are in eV
    lookup_table = {
        1: 18.9,
        2: 42.0,
        3: 38.0,
        4: 60.0, 
        6: 78.0,
        7: 85.0,
        8: 89.0,
        10: 131.0,
        13: 163.0
    }
    
    # Check to see if Z is in our table
    I_a = lookup_table.get(Z)
    
    # If I is not in the table, calculate it
    # Use Anderson Equation 2.33
    if I_a is None:
        I_a = 9.73 * Z + 58.8 * Z ** -0.19
    
    # Convert I from eV to MeV
    I_a = I_a * 10**-6.0
    
    return I_a

In [12]:
def beta_squared(T, m):
    """ Gives value of beta^2 for a given Mass (MeV/c^2) and Kinetic Energy (MeV)
    """
    numerator = T * (T + 2 * m)
    denominator = (T + m) ** 2
    return numerator / denominator

In [13]:
def gamma_squared(T, m):
    """ Gives value of gamma^2 for a given mass (MeV/c^2) and Kinetic Energy (MeV)
    """
    return 1.0 / (1 - beta_squared(T, m)) ** 0.5

In [14]:
def S_c(incident, target, T, M_m, M_b):
    """ Returns the stopping power in MeV/cm
        T in MeV
        density in g/cm^3
        Output in MeV/cm
    """
    # Currently the incident and target are specified in Z number. incident is assumed to be fully ionized
    z = incident
    Z = target
    
    # Convert the mass of the beam particles into MeV
    M_b = M_b * m_to_e
    
    first = 4 * (z ** 2) * pi * (r_e ** 2) * m_e
    second = N_A * Z / M_m # TODO: Take M_m from a Pyne material
    third = 1.0 / beta_squared(T, M_b)
    logpart = (2 * m_e * beta_squared(T, M_b) * gamma_squared(T, M_b)) / (I(Z))
    fourth = math.log(logpart) - beta_squared(T, M_b) + beta_squared(T, M_b)
    
    print json.dumps(locals(), indent=1)
    print gamma_squared(T, M_b)
    print beta_squared(T, M_b)
    
    return first * second * third * fourth

In [15]:
S_c(14, 13, 500 * 28, 26.98154, 28.0)

{
 "z": 14, 
 "logpart": 5555.514480755896, 
 "third": 1.734393217150448, 
 "M_b": 26081.8346712, 
 "incident": 14, 
 "second": 2.901533090438871e+23, 
 "T": 14000, 
 "fourth": 8.622546313582793, 
 "target": 13, 
 "Z": 13, 
 "M_m": 26.98154, 
 "first": 9.994234214414889e-23
}
1.53677205521
0.576570520521


433.6707717870548

### Calculate the energy lost by a charged particle with a given Z, mass, and K.E. through a given thickness of material (materials listed below) ###

Elements:
- Z = 1, 6, 7, 8, 11, 12, 13, 14, 18, 26, 29, 50, 74, 79, 82, 92

Composite Materials:
- water (liquid)
- concrete
- air
- adipose tissue

In [16]:
from pyne import nuc_data
from pyne.material import MaterialLibrary
import pyne.material

In [17]:
class Material(object):
    """ Placeholder for the target/shielding materials. Should upgrade these to Pyne materials
    """
    def __init__(self, **kwargs):
        self.__dict__ = kwargs
    
    def set_excitation(I):
        self.I = I
    
    def set_z(z):
        self.Z = z

### Estimate the range of a charged particle through a given material by stepping through the material until the stopping power approaches zero (note, if you take the energy down to the lower limit of 2-3 MeV/nucleon, you can stop the calculation there--there isn't much range left after that energy) ###

First define the step size:

Thoughts:
- define with relationship to density? This would correct for state of matter
    - Could also use "S/rho" relationships from notes
    - maybe just use 1/rho in cm or something
- take a guess/iterate approach? as in, select an arbitrary thickness (1/2 of total or something), then try smaller and smaller fractions until the range converges?
    - Spawn multiple threads to speed up calculations? one per thickness?
        - How to even do this? would need to know E_n to calculate E_(n+1)

In [2]:
def transit(beam, target, thickness):
    """ Increments the energy of an incident particle (beam) after transiting through thickness of material target
        beam should be of type Beam
        target should be of type Material
        thickness is in cm
        
        thickness should be chose carefully
    """
    # First calculate specific mass stopping power
    
    # Then multiply specific mass stopping power by density of material
    
    # Then multiply by thickness
    
    # Subtract that result from the beam energy, beam.T

### Using geometric cross sections, estimate the fraction of the fluence undergoing a nuclear interaction as it goes through a given thickness of material ###

In [19]:
def geometric_cross_section(A1, A2):
    """ Gives the geometric cross section for the collision of two nuclei
        Inputs: A1, A2--mass number of nuclei
        Output: sigma--microscopic cross section (barns)
    """
    # First calculate "radius"
    def R(A):
        r_0 = 1.4 * 10 ** -13.0 # cm
        return r_0 * A ** (1.0 / 3.0)
    return math.pi * (R(A1) + R(A2)) ** 2

The fluence at distance $ x $ through a material is $$ \Phi = \Phi_0 e^{\Sigma x} = \Phi_0 e ^ {\sigma N x} $$